In [70]:
import numpy as np
import pandas as pd
import re,math
from sklearn import preprocessing
from scipy.stats import pearsonr
import matplotlib.pyplot as plt



data1 = pd.read_csv(r'2022_02_23_07_33_27.csv',encoding='gbk')[1:]
data2 = pd.read_excel(r'选择你所在家乡城市 描述统计.xlsx')
data3 = pd.read_excel(r'学校以及学院 描述统计.xlsx')
data4 = pd.read_excel(r'W020211027623974108131.xls')
data5 = pd.read_excel(r'中国一二三四五线城市名单.xlsx')
data6 = pd.read_excel(r'一流大学.xlsx')
data7 = pd.read_excel(r'兼职倾向.xlsx')
# 去空
for i in range(len(data4)):
    if pd.isnull(data4["Unnamed: 5"][i]):
        data4 = data4.drop(i)


In [71]:
data1

,作答ID,用户ID,用户名,开始时间,结束时间,作答时长(秒),作答渠道,发布ID,问卷发布名称,IP,...,在您看来，您的大学着重培养学生哪些方面的能力？-提升眼界,在您看来，您的大学着重培养学生哪些方面的能力？-专业知识,在您看来，您的大学着重培养学生哪些方面的能力？-专业技能应用,在您看来，您的大学着重培养学生哪些方面的能力？-德智体美劳全面发展,在您看来，您的大学着重培养学生哪些方面的能力？-其他能力,在您看来，招聘单位通常更青睐具备哪些特点的人才？-专业理论扎实,在您看来，招聘单位通常更青睐具备哪些特点的人才？-专业技能丰富,在您看来，招聘单位通常更青睐具备哪些特点的人才？-涉猎专业多元,在您看来，招聘单位通常更青睐具备哪些特点的人才？-较强的应变能力,您对未来工作的期望：
1,M6BPlr8M,yevljzV7,cdm****cB5Fs,2022-2-18 14:26,2022-2-18 14:29,229,1,NriUJr,大学生兼职调查,120.206.21.46,...,1,1,1,0,0,0,1,0,1,满足基本生活需要（满足日常基本的衣食住行和睡眠）
2,7XAkgPGL,37KOkkO7,cdm****5Wpzy,2022-2-17 22:37,2022-2-17 22:40,225,1,NriUJr,大学生兼职调查,112.47.113.139,...,1,1,0,0,0,1,1,0,0,有归属感和社交（单位氛围暖心，与同事关系融洽）
3,M9X0AO4e,veEAKVve,cdm****xbfAi,2022-2-17 20:37,2022-2-17 20:40,178,1,NriUJr,大学生兼职调查,49.52.96.48,...,1,1,0,0,0,1,1,1,0,满足安全需要（工作稳定，很少有焦虑）
4,MdP1j3QM,me3dVW2M,cdm****3GOx2,2022-2-17 20:31,2022-2-17 20:34,202,1,NriUJr,大学生兼职调查,120.231.61.11,...,1,0,1,0,0,0,1,0,1,满足自我实现需要（激发潜能，实现自己的价值）
5,Mx2zWy0L,27an5awL,cdm****vzAcC,2022-2-17 20:16,2022-2-17 20:21,343,1,NriUJr,大学生兼职调查,113.82.142.74,...,0,1,1,0,0,1,0,0,1,满足自我实现需要（激发潜能，实现自己的价值）
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
596,MdP10VAM,0ey0blvL,cdm****zf4iV,2022-2-16 13:03,2022-2-16 13:06,189,1,NriUJr,大学生兼职调查,60.220.49.136,...,0,1,1,0,0,1,1,0,1,受到尊重（工作体面有尊严
597,7qXd386M,2Md1dvA7,小*叽～,2022-2-16 13:03,2022-2-16 13:06,196,1,NriUJr,大学生兼职调查,111.179.184.140,...,0,1,1,1,0,0,1,0,1,满足自我实现需要（激发潜能，实现自己的价值）
598,Mr5ngqBe,0eyPgEvL,cdm****PptQd,2022-2-16 13:03,2022-2-16 13:06,173,1,NriUJr,大学生兼职调查,112.38.54.174,...,0,1,0,0,0,1,0,0,1,满足自我实现需要（激发潜能，实现自己的价值）
599,7YpQDvWM,27ap49OM,cdm****XjksY,2022-2-16 13:04,2022-2-16 13:06,133,1,NriUJr,大学生兼职调查,42.178.82.103,...,1,1,1,0,0,1,1,0,0,满足安全需要（工作稳定，很少有焦虑）


In [72]:
bkyx = np.array(data4[data4["Unnamed: 5"] == "本科"]["Unnamed: 1"])
zkyx = np.array(data4[data4["Unnamed: 5"] == "专科"]["Unnamed: 1"])
ytemp = []        
bktemp = []
zktemp = []
ccp = []
for index, row in data1.iterrows():
    try:
        tp = row["学校以及学院"].split("大学")[0] + "大学"
    except:
        tp = row["学校以及学院"]
    if tp in bkyx and tp not in np.array(data6[0]):
        bktemp.append(np.array(row))
        row["学校以及学院"] = 2
    elif tp in np.array(data6[0]):
        ytemp.append(np.array(row))
        row["学校以及学院"] = 1
    else:
        zktemp.append(np.array(row))
        row["学校以及学院"] = 3
    ccp.append(row["学校以及学院"])
data1["学校以及学院"] = ccp
# data1["学校以及学院"]
for i,j in zip([ytemp, bktemp, zktemp],["双一流大学", "普通本科大学", "专科大学"]):
    pd.DataFrame(i, columns=data1.columns).to_csv(f"./大学描述/{j}.csv",encoding="utf_8_sig")

In [73]:
tpc = data5.copy()
tem = np.array(data5["城市链接"])
temp = []
temp2 = []
for i in tem:
    if len(i.split("：")) == 2:
        temp.append(i.split("：")[1])
        temp2.append(i.split("：")[0])
    else:
        temp.append(i)
        temp2.append(i)
tpc["城市链接"] = pd.DataFrame(temp)
tpc = pd.concat([tpc, pd.DataFrame(temp2)], axis=1).drop(["Unnamed: 6", "Unnamed: 4", "全国共计337个地级以上城市（2019年5月）：" ], axis=1)
yxcity = np.array(tpc[tpc["城市分级"] == "一线城市"]["城市链接"])
xyxcity = np.array(tpc[tpc["城市分级"] == "新一线城市"]["城市链接"])
erxcity = np.array(tpc[tpc["城市分级"] == "二线城市"]["城市链接"])
sanxcity = np.array(tpc[tpc["城市分级"] == "三线城市"]["城市链接"])
sixcity = np.array(tpc[tpc["城市分级"] == "四线城市"]["城市链接"])
wuxcity = np.array(tpc[tpc["城市分级"] == "五线城市"]["城市链接"])
#yxtemp = []
#xyxtemp = []
#erxtemp = []
#sanxtemp = []
#sixtemp = []
#wuxtemp = []
tp = []
for index, row in data1.iterrows():
#     tp1 = row["类别"].split("，")[0]
#     tp2 = row["类别"].split("，")[1]
    tp2 = row["城市"]
    if tp2 in yxcity:
        row["城市"] = 1
        #yxtemp.append(np.array(row))
        tp.append(1)
    elif tp2 in xyxcity:
        row["城市"] = 2
        tp.append(2)
        #xyxtemp.append(np.array(row))
    elif tp2 in erxcity:
        row["城市"] = 3
        tp.append(3)
        #erxtemp.append(np.array(row))
    elif tp2 in sanxcity:
        row["城市"] = 4
        tp.append(4)
        #sanxtemp.append(np.array(row))
    elif tp2 in sixcity:
        row["城市"] = 5
        tp.append(5)
        #sixtemp.append(np.array(row))
    elif tp2 in wuxcity:
        row["城市"] = 6
        tp.append(6)
        #wuxtemp.append(np.array(row))
    else:
        tp.append(1)
        row["城市"] = 1
        # print(row)
        # row["城市"] = 7
#         errotemp.append(np.array(row))
data1["城市"] = tp
data1["城市"]

1      3
2      3
3      1
4      5
5      5
      ..
596    6
597    5
598    5
599    6
600    1
Name: 城市, Length: 600, dtype: int64

In [74]:
tp = []
for index, row in data1.iterrows():
    if row["年级"] == "大一":
        row["年级"] = 1
        tp.append(1)
    elif row["年级"] == "大二":
        row["年级"] = 2
        tp.append(2)
    elif row["年级"] == "大三":
        row["年级"] = 3
        tp.append(3)
    elif row["年级"] == "大四":
        row["年级"] = 4
        tp.append(4)
    elif row["年级"] == "研一":
        row["年级"] = 5
        tp.append(5)
    elif row["年级"] == "研二":
        row["年级"] = 6
        tp.append(6)
    elif row["年级"] == "研三":
        row["年级"] = 7
        tp.append(7)
    elif row["年级"] == "博士":
        row["年级"] = 8
        tp.append(8)
    else:
        print(row["年级"])
data1["年级"] = tp
data1["年级"]

1      1
2      3
3      6
4      2
5      3
      ..
596    2
597    1
598    3
599    1
600    3
Name: 年级, Length: 600, dtype: int64

In [75]:
data1

,作答ID,用户ID,用户名,开始时间,结束时间,作答时长(秒),作答渠道,发布ID,问卷发布名称,IP,...,在您看来，您的大学着重培养学生哪些方面的能力？-提升眼界,在您看来，您的大学着重培养学生哪些方面的能力？-专业知识,在您看来，您的大学着重培养学生哪些方面的能力？-专业技能应用,在您看来，您的大学着重培养学生哪些方面的能力？-德智体美劳全面发展,在您看来，您的大学着重培养学生哪些方面的能力？-其他能力,在您看来，招聘单位通常更青睐具备哪些特点的人才？-专业理论扎实,在您看来，招聘单位通常更青睐具备哪些特点的人才？-专业技能丰富,在您看来，招聘单位通常更青睐具备哪些特点的人才？-涉猎专业多元,在您看来，招聘单位通常更青睐具备哪些特点的人才？-较强的应变能力,您对未来工作的期望：
1,M6BPlr8M,yevljzV7,cdm****cB5Fs,2022-2-18 14:26,2022-2-18 14:29,229,1,NriUJr,大学生兼职调查,120.206.21.46,...,1,1,1,0,0,0,1,0,1,满足基本生活需要（满足日常基本的衣食住行和睡眠）
2,7XAkgPGL,37KOkkO7,cdm****5Wpzy,2022-2-17 22:37,2022-2-17 22:40,225,1,NriUJr,大学生兼职调查,112.47.113.139,...,1,1,0,0,0,1,1,0,0,有归属感和社交（单位氛围暖心，与同事关系融洽）
3,M9X0AO4e,veEAKVve,cdm****xbfAi,2022-2-17 20:37,2022-2-17 20:40,178,1,NriUJr,大学生兼职调查,49.52.96.48,...,1,1,0,0,0,1,1,1,0,满足安全需要（工作稳定，很少有焦虑）
4,MdP1j3QM,me3dVW2M,cdm****3GOx2,2022-2-17 20:31,2022-2-17 20:34,202,1,NriUJr,大学生兼职调查,120.231.61.11,...,1,0,1,0,0,0,1,0,1,满足自我实现需要（激发潜能，实现自己的价值）
5,Mx2zWy0L,27an5awL,cdm****vzAcC,2022-2-17 20:16,2022-2-17 20:21,343,1,NriUJr,大学生兼职调查,113.82.142.74,...,0,1,1,0,0,1,0,0,1,满足自我实现需要（激发潜能，实现自己的价值）
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
596,MdP10VAM,0ey0blvL,cdm****zf4iV,2022-2-16 13:03,2022-2-16 13:06,189,1,NriUJr,大学生兼职调查,60.220.49.136,...,0,1,1,0,0,1,1,0,1,受到尊重（工作体面有尊严
597,7qXd386M,2Md1dvA7,小*叽～,2022-2-16 13:03,2022-2-16 13:06,196,1,NriUJr,大学生兼职调查,111.179.184.140,...,0,1,1,1,0,0,1,0,1,满足自我实现需要（激发潜能，实现自己的价值）
598,Mr5ngqBe,0eyPgEvL,cdm****PptQd,2022-2-16 13:03,2022-2-16 13:06,173,1,NriUJr,大学生兼职调查,112.38.54.174,...,0,1,0,0,0,1,0,0,1,满足自我实现需要（激发潜能，实现自己的价值）
599,7YpQDvWM,27ap49OM,cdm****XjksY,2022-2-16 13:04,2022-2-16 13:06,133,1,NriUJr,大学生兼职调查,42.178.82.103,...,1,1,1,0,0,1,1,0,0,满足安全需要（工作稳定，很少有焦虑）


In [76]:
data1.to_csv("600.csv",encoding="utf_8_sig")

In [58]:
# pd.DataFrame(errotemp).to_csv("0303.csv",encoding="utf_8_sig")

In [59]:
tp = []
for index, row in data1.iterrows():
    if row["您从事过几份兼职？"] == "1份":
        row["您从事过几份兼职？"] = 1
    elif row["您从事过几份兼职？"] == "2-3份":
        row["您从事过几份兼职？"] = 3
    elif row["您从事过几份兼职？"] == "4-7份":
        row["您从事过几份兼职？"] = 6
    elif row["您从事过几份兼职？"] == "10份以上":
        row["您从事过几份兼职？"] = 10
    else:
        continue
    tp.append(row)
data1 = pd.DataFrame(tp, columns=data1.columns)

In [60]:
tp = []
for index, row in data1.iterrows():
    if row["您的父母对于兼职的态度是什么样的？"] == "支持":
        row["您的父母对于兼职的态度是什么样的？"] = 1
    elif row["您的父母对于兼职的态度是什么样的？"] == "中立":
        row["您的父母对于兼职的态度是什么样的？"] = 0
    elif row["您的父母对于兼职的态度是什么样的？"] == "反对":
        row["您的父母对于兼职的态度是什么样的？"] = -1
    else:
        continue
    tp.append(row)
data1 = pd.DataFrame(tp, columns=data1.columns)

In [61]:
max_abs_scaler = preprocessing.MaxAbsScaler()
jz = np.array(data1["您从事过几份兼职？"])
nj = np.array(data1["年级"])
cs = np.array(data1["城市"])
xx = np.array(data1["学校以及学院"])
td = np.array(data1["您的父母对于兼职的态度是什么样的？"])

In [62]:
data1["学校以及学院"]

2      1
3      1
4      1
5      2
6      2
      ..
595    1
596    1
597    3
599    1
600    2
Name: 学校以及学院, Length: 549, dtype: int64

In [63]:
abs(pd.DataFrame(max_abs_scaler.fit_transform([jz,nj])).T.corr("spearman"))

,0,1
0,1.000000,0.535323
1,0.535323,1.000000


In [64]:
abs(pd.DataFrame(max_abs_scaler.fit_transform([jz,cs])).T.corr("spearman"))

,0,1
0,1.000000,0.644069
1,0.644069,1.000000


In [65]:
abs(pd.DataFrame(max_abs_scaler.fit_transform([jz,xx])).T.corr("spearman"))

,0,1
0,1.000000,0.471818
1,0.471818,1.000000


In [66]:
# abs(pd.DataFrame(max_abs_scaler.fit_transform([jz,td])).T.corr("spearman"))

In [68]:
data1.shape

(549, 148)